In [8]:
import pandas as pd

# Path to the input Excel file
file_path = r"D:\climate change\Tutorial_Climate Data_2025-26.xlsx"

# Output file paths with specified location
monthly_output_path = r"D:\climate change\monthly_averages.xlsx"
yearly_output_path = r"D:\climate change\yearly_averages.xlsx"

# Load all sheets
excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names

# Prepare writers for output files
monthly_writer = pd.ExcelWriter(monthly_output_path, engine='openpyxl', datetime_format='YYYY-MM-DD')
yearly_writer = pd.ExcelWriter(yearly_output_path, engine='openpyxl', datetime_format='YYYY-MM-DD')

for sheet in sheet_names:
    # Read the sheet without assuming headers
    df = pd.read_excel(file_path, sheet_name=sheet, header=None)
    
    # Extract lons (row 1, starting from column 1)
    lons = df.iloc[1, 1:]
    
    # Extract lats (row 2, starting from column 1)
    lats = df.iloc[2, 1:]
    
    # Extract dates (column 0, starting from row 3)
    dates = df.iloc[3:, 0]
    
    # Extract data values (starting from row 3, column 1)
    data = df.iloc[3:, 1:]
    
    # Create station names (assuming 23 stations based on data)
    num_stations = data.shape[1]
    station_names = [f"Station-{i}" for i in range(1, num_stations + 1)]
    
    # Create the data DataFrame with dates as index
    data_df = pd.DataFrame(data.values, index=pd.to_datetime(dates), columns=station_names)
    
    # Compute monthly averages (mean of daily values per month)
    monthly_avg = data_df.resample('ME').mean()
    
    # Compute yearly averages (mean of daily values per year)
    yearly_avg = data_df.resample('YE').mean()
    
    # Prepare output DataFrames with Lon and Lat rows
    # For monthly
    monthly_out = pd.DataFrame(index=['Lon', 'Lat'] + list(monthly_avg.index), columns=station_names)
    monthly_out.loc['Lon'] = lons.values
    monthly_out.loc['Lat'] = lats.values
    monthly_out.loc[monthly_avg.index] = monthly_avg.values
    
    # For yearly
    yearly_out = pd.DataFrame(index=['Lon', 'Lat'] + list(yearly_avg.index), columns=station_names)
    yearly_out.loc['Lon'] = lons.values
    yearly_out.loc['Lat'] = lats.values
    yearly_out.loc[yearly_avg.index] = yearly_avg.values
    
    # Write to output files
    monthly_out.to_excel(monthly_writer, sheet_name=f"{sheet}_monthly")
    yearly_out.to_excel(yearly_writer, sheet_name=f"{sheet}_yearly")

# Close the writers to save the files
monthly_writer.close()
yearly_writer.close()

print(f"Monthly averages saved to {monthly_output_path}")
print(f"Yearly averages saved to {yearly_output_path}")

Monthly averages saved to D:\climate change\monthly_averages.xlsx
Yearly averages saved to D:\climate change\yearly_averages.xlsx
